In [54]:
# Save the link to the data, to later be able to delete it from the directory to save storage
link_to_data = f"https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt" 
filename = "tiny_shakespeare.txt"

In [55]:
import torch
import torch.nn as nn 
from torch.nn import functional as F

import requests

In [56]:
# Get data

# r = requests.get(link_to_data)

# with open(filename, "w") as f:
#     f.write(r.text)

In [57]:
# Read the data
with open(filename, "r", encoding="utf-8") as f:
    text = f.read()

In [58]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [59]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

####  Create encoding and decoding functions

In [60]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(f"{vocab_size=}")



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size=65


In [61]:
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for i, s in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [62]:
# Try the encoder and decoder
test_line = "hello there"
print(encode(test_line))
print(decode(encode(test_line)))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hello there


### Create train and test splits

In [63]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [64]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print(f"{train_data.shape = }")
print(f"{val_data.shape = }")

train_data.shape = torch.Size([1003854])
val_data.shape = torch.Size([111540])


In [65]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    label = y[t]
    print(f"When input is {context.tolist()} the label is {label}")

When input is [18] the label is 47
When input is [18, 47] the label is 56
When input is [18, 47, 56] the label is 57
When input is [18, 47, 56, 57] the label is 58
When input is [18, 47, 56, 57, 58] the label is 1
When input is [18, 47, 56, 57, 58, 1] the label is 15
When input is [18, 47, 56, 57, 58, 1, 15] the label is 47
When input is [18, 47, 56, 57, 58, 1, 15, 47] the label is 58


In [50]:
data = torch.tensor([i for i in range(10)])

In [73]:
ix = torch.randint(len(data) - block_size, (4, ))
x = torch.stack([data[i: i+block_size] for i in ix])
y = torch.stack([data[i+1: i+block_size+1] for i in ix])

In [80]:
x

tensor([[51, 39, 63,  1, 42, 43, 52, 63],
        [35, 39, 47, 50, 47, 52, 45,  1],
        [ 5, 42,  1, 50, 53, 60, 43,  1],
        [52, 45,  1, 44, 56, 53, 51,  1]])

In [53]:
[data[i: i+block_size] for i in ix]

[tensor([0, 1, 2, 3, 4, 5, 6, 7]),
 tensor([1, 2, 3, 4, 5, 6, 7, 8]),
 tensor([0, 1, 2, 3, 4, 5, 6, 7]),
 tensor([1, 2, 3, 4, 5, 6, 7, 8])]

In [45]:
ix

tensor([357341, 442374, 202931, 950443])

In [32]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split: str = "train"):
    data = train_data if split == "val" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])

    return x, y

xb, yb = get_batch()
print(f"{xb.shape = }")
print(f"{yb.shape = }")

print("\ninputs:")
print(xb)
print("targets:")
print(yb)

print("---------")
for i in range(batch_size):
    for j in range(block_size):
        context = xb[i, :j+1]
        target = yb[i, j]

        print(f"When the context is {context.tolist()}, the target is {target}")

tensor([29535, 38737, 81972, 56048])
xb.shape = torch.Size([4, 8])
yb.shape = torch.Size([4, 8])

inputs:
tensor([[ 6,  1, 52, 53, 58,  1, 58, 47],
        [ 6,  1, 54, 50, 39, 52, 58, 43],
        [ 1, 58, 46, 47, 57,  1, 50, 47],
        [ 0, 32, 46, 43, 56, 43,  1, 42]])
targets:
tensor([[ 1, 52, 53, 58,  1, 58, 47, 50],
        [ 1, 54, 50, 39, 52, 58, 43, 58],
        [58, 46, 47, 57,  1, 50, 47, 60],
        [32, 46, 43, 56, 43,  1, 42, 53]])
---------
When the context is [6], the target is 1
When the context is [6, 1], the target is 52
When the context is [6, 1, 52], the target is 53
When the context is [6, 1, 52, 53], the target is 58
When the context is [6, 1, 52, 53, 58], the target is 1
When the context is [6, 1, 52, 53, 58, 1], the target is 58
When the context is [6, 1, 52, 53, 58, 1, 58], the target is 47
When the context is [6, 1, 52, 53, 58, 1, 58, 47], the target is 50
When the context is [6], the target is 1
When the context is [6, 1], the target is 54
When the contex

In [22]:
x, y = get_batch()

In [87]:
y

tensor([[39, 63,  1, 42, 43, 52, 63,  1],
        [39, 47, 50, 47, 52, 45,  1, 53],
        [42,  1, 50, 53, 60, 43,  1, 44],
        [45,  1, 44, 56, 53, 51,  1, 58]])

# Create a simple Bigram baseline model

In [26]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) 
        
    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(f"{loss.item() = }")


print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
loss.item() = 4.4912800788879395

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [27]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [42]:
# Train the model
batch_size = 32
for _ in range(10000):
    # Sample a batch of data
    xb, yb = get_batch()

    # Make predictions
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

NameError: name 'model' is not defined

In [43]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=200)[0].tolist()))

NameError: name 'model' is not defined

#### The mathematical trick in self-attention

In [30]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2

x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [31]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [32]:
wei = torch.tril(torch.ones((T, T)))
wei = wei / torch.sum(wei, dim=-1, keepdim=True)
xbow2 = wei @ x

In [33]:
torch.allclose(xbow, xbow2)

True

In [34]:
# 3rd version - using softmax
tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)

xbow3 = wei @ x

In [35]:
torch.allclose(xbow, xbow3)

True

In [36]:
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(1, 10, (3, 2)).float()
c = a @ b
print("a = ")
print(a)
print("b = ")
print(b)
print("c = ")
print(c)


a = 
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b = 
tensor([[7., 6.],
        [8., 5.],
        [1., 3.]])
c = 
tensor([[16., 14.],
        [16., 14.],
        [16., 14.]])


In [37]:
# The trick is to use triangular matrices in order to not allow looking at the future data
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3)) # Create a lower triangle
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(1, 10, (3, 2)).float()
c = a @ b
print("a = ")
print(a)
print("b = ")
print(b)
print("c = ")
print(c)

a = 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b = 
tensor([[7., 6.],
        [8., 5.],
        [1., 3.]])
c = 
tensor([[7.0000, 6.0000],
        [7.5000, 5.5000],
        [5.3333, 4.6667]])


## Self-Attention Mechanism

In [58]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# Single head self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False) 
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
wei = q @ k.transpose(-2, -1)

tril = torch.tril(torch.ones(T, T)) 
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = wei * head_size **-0.5 # Scaling down to preserve variance
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [59]:
out[0]

tensor([[-1.5713e-01,  8.8009e-01,  1.6152e-01, -7.8239e-01, -1.4289e-01,
          7.4676e-01,  1.0068e-01, -5.2395e-01, -8.8726e-01,  1.9067e-01,
          1.7616e-01, -5.9426e-01, -4.8124e-01, -4.8599e-01,  2.8623e-01,
          5.7099e-01],
        [ 4.3974e-01, -1.4227e-01, -1.3157e-01,  2.8895e-03, -1.3222e-01,
          6.6093e-04, -2.7904e-01, -2.2676e-01, -2.8723e-01,  5.7456e-01,
          5.6053e-01, -2.5208e-01,  9.7243e-02,  1.0771e-01,  3.0455e-02,
          1.0727e+00],
        [ 4.3615e-01, -6.6358e-02, -2.9296e-01,  7.4315e-02,  5.4381e-02,
         -7.0388e-02, -6.8984e-02, -8.2153e-02, -2.9377e-01, -5.8952e-02,
          3.5887e-01, -2.3088e-03, -1.8212e-01, -3.6143e-02, -6.7189e-02,
          1.1412e+00],
        [ 4.2068e-01, -1.0619e-01, -2.9984e-01,  5.2820e-02,  2.0077e-01,
         -1.6048e-01, -3.5710e-02, -8.3110e-02, -1.7919e-01,  7.7992e-02,
          1.2719e-01,  2.2611e-02, -5.1810e-02,  7.4466e-02,  1.8131e-01,
          8.4463e-01],
        [ 3.9499e-01